Next Text generator
This model is trained from from text taken from wikipedia (space theme )  so output are biased to that.

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import re, requests
from collections import Counter
from torch.utils.data import TensorDataset, DataLoader

In [5]:
import requests       #use download file
url = "https://raw.githubusercontent.com/kamalmalikofficial/next_word_finder/refs/heads/main/solarsys%20%26%20Earth.txt"
# 👆 is the text file from my github

text = requests.get(url).text
text = text * 15  # increase the same text count for confident learning

print("Text length: ",    f"{len(text)}"   ,"characters")
print("This is approx ",  f"{len(text)/1024:.1f}"  ,"KB")
print(text[:500]) # use to print first 500 word just for confirmation



Text length:  2182950 characters
This is approx  2131.8 KB
Have you ever looked up at the night sky and wondered what's out there? Space is absolutely 
massive, beyond anything we can really imagine. The universe contains billions of galaxies, 
and each galaxy has billions of stars. It's mind-blowing when you think about it.
Our solar system is pretty amazing too. The Sun sits at the center, and it's huge compared 
to Earth. You could fit over a million Earths inside the Sun. It's basically a giant ball 
of burning gas, mostly hydrogen and helium. 


In [6]:
!pip install torchtext


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.6 MB/s eta 0:00:00


In [7]:
import re
from collections import Counter

# simple tokenizer (like torchtext basic_english)
def simple_tokenizer(text):
    return re.findall(r"\b\w+\b", text.lower())

tokens = simple_tokenizer(text)

# build vocab
def build_vocab(tokens, max_size=2000):
    counts = Counter(tokens)
    most_common = counts.most_common(max_size - 1)
    vocab = {word: i + 1 for i, (word, _) in enumerate(most_common)}
    vocab["<unk>"] = 0
    return vocab

vocab = build_vocab(tokens)

# encode text
def encode(tokens, vocab):
    return [vocab.get(token, 0) for token in tokens]

encoded = encode(tokens, vocab)

# create input-output pairs (predict next token)
seq_len = 20
data = []
for i in range(len(encoded) - seq_len):
    x = encoded[i:i + seq_len]
    y = encoded[i + seq_len]
    data.append((x, y))


In [8]:
for i, (word, idx) in enumerate(vocab.items()):
    print(f"{word:10} → {idx}")
    if i == 20:  # just first 20 entries
        break


the        → 1
of         → 2
and        → 3
is         → 4
to         → 5
earth      → 6
a          → 7
in         → 8
s          → 9
sun        → 10
are        → 11
solar      → 12
it         → 13
as         → 14
with       → 15
that       → 16
system     → 17
from       → 18
by         → 19
at         → 20
its        → 21


In [9]:
# use to do train_test_split to check validation and accuracy
from torch.utils.data import random_split, DataLoader, TensorDataset

# suppose like data is list of (x, y) pairs
X = torch.tensor([x for x, _ in data], dtype=torch.long)
Y = torch.tensor([y for _, y in data], dtype=torch.long)

dataset = TensorDataset(X, Y)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])  # to superrandonly split the data

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)


In [10]:

class TinyNextWordModel(nn.Module):
    def __init__(self, vocab_size=2000, embed_dim=32, hidden_dim=64):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

vocab_size = min(len(vocab), 2000)
model = TinyNextWordModel(vocab_size)

print("Total trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))


Total trainable params: 212816


In [11]:
criterion = nn.CrossEntropyLoss()  # instead of mse cause treats word id like no.

optimizer = optim.Adam(model.parameters(), lr=1e-3) # adam is the most commanly used optimizer in dl
epochs = 20   # this is the no. of epochs or no. of time we improve our result or decrese out loss function
for epoch in range(epochs):
    # main training code
    model.train()
    total_train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # for validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            out = model(xb)
            loss = criterion(out, yb)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")


Epoch 1: Train Loss = 4.8248, Val Loss = 3.9268


In [ ]:
# save the entire model into pkl file cause the above code take 20 min to train
torch.save(model, "trained_model.pkl")
print("model saved")


In [ ]:

# use to do the dict mapping inverse
inv_vocab = {idx: word for word, idx in vocab.items()}

def predict_topk(model, text_input, seq_len=20, top_k=5):
    model.eval()
    tokens = simple_tokenizer(text_input)  # same tokenizer as training
    encoded = [vocab.get(t, 0) for t in tokens[-seq_len:]]
    if len(encoded) < seq_len:
        encoded = [0]*(seq_len - len(encoded)) + encoded
    x = torch.tensor(encoded).unsqueeze(0)

    with torch.no_grad():
        out = model(x)
        probs = F.softmax(out, dim=1)
        top_probs, top_idxs = probs.topk(top_k, dim=1)

    results = [(inv_vocab.get(idx.item(), "unknown"), prob.item())
               for idx, prob in zip(top_idxs[0], top_probs[0])]
    return results

In [ ]:
import torch.nn.functional as F


In [ ]:
preds = predict_topk(model, " black ", top_k=5)
for word, prob in preds:
    print(f"{word:10s} — {prob:.2f}")


In [ ]:
torch.save(vocab, "vocab.pkl")


In [ ]:
torch.save(inv_vocab, "inv_vocab.pkl")